# Definitions

Suppose we have a raw RASS image $\mathbf A$, with pixel values $A_{ij}$ and size $N \times N$, covering a galaxy group with redshift $z_{\rm grp}$:
$$ \mathbf{A} = \pmatrix{A_{00} & \dots  & A_{0N} \\ \vdots & \ddots & \vdots \\A_{N0} & \dots & A_{NN}} ,\,\, {\rm size}\, N\times N $$
<br>
The goal of the image scaling procedure is to make a new, scaled image $\mathbf{A'}$, which:
- changes the angular size of objects in the original image such that $\mathbf{A'}$ corresponds in physical size to a larger redshift $z_{\rm max}$.
- maintains the resolution of the original (45''/px for RASS)
<br>

To accomplish this, we need a scaling factor $s$ to scale the images:
$$ s = \frac{z_{\rm grp}}{z_{\rm max}}. $$

<br>

The scaled image $\mathbf{A'}$ constructed from $s$ and $\mathbf A$ will occupy fewer pixels to cover the same physical size:

$$ \mathbf{A}' = \pmatrix{0 & \dots  & 0 \\ \vdots & A_{kl}' & \vdots \\0 & \dots & 0},\,\, {{\rm size}\, N\times N}. $$

Therefore, there is not a simple one-to-one mapping between pixels in $\mathbf{A}$ and non-zero pixels in $\mathbf{A'}$. The mapping function we use (applied through `scipy.ndimage.geometric_transform`) is given by:

$$ \mathcal{G}: (i,j)=\left(\frac{2k + Ns - N}{2s}, \frac{2l + Ns - N}{2s}\right), $$

which describes the position $(i,j)$ in the original image as a function of the position $(k,l)$ in the scaled image.

## Examples of $\mathcal{G}$<br>
If we apply $\mathcal{G}$ to the central pixel in $\mathbf{A'}$, it will return the same value. The center pixel is located at the same place in both images.

In [ ]:
def Gmap(kk,ll,s):
    return ((2*kk+N*s-N)/(2*s),(2*ll+N*s-N)/(2*s))

In [ ]:
N, s = 300, 0.5
Gmap(N//2, N//2, s)

In [ ]:
N, s = 300, 0.2
Gmap(N//2, N//2, s)

The corners of images should have the most drastic change from $(i,j)$ to $(k,l)$ The corner pixel $(0,0)$ in $\mathbf{A}$ corresponds to $(75,75)$ in $\mathbf A'$ for $N=300$, $s=0.5$. Likewise, a pixel in $\mathbf A$ at $(300,300)$ corresponds to $(225,225)$ in $\mathbf A'$.

In [ ]:
N, s = 300, 0.5
Gmap(75,75, s)

In [ ]:
N, s = 300, 0.5
Gmap(225, 225, s)

## Determing Pixel Values in $A'$

Given that the number of pixels in $\mathbf{A}$ does not correspond one-to-one with non-zero pixels in $\mathbf{A'}$, the pixel values must be combined in $\mathbf A'$ if it will contain the same information. The pixel values in $\mathbf A'$ are determined using a spline fit of $A$ given the pixel position mapping.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import geometric_transform

def mapping(coords,s):
    return ((2*coords[0]+N*s-N)/(2*s),(2*coords[1]+N*s-N)/(2*s))

NN = 300
xx=np.arange(0,NN,1)
counts = 5*np.exp(-1*(xx-NN//2)**2./(500))
image = counts*counts[:,None]
scaledimage = geometric_transform(image,mapping,extra_keywords={'s':0.5})

fig,axs=plt.subplots(ncols=2)
axs[0].imshow(image)
axs[1].imshow(scaledimage)
plt.show()

If the geometric transformation conserves flux then it will satisfy
$$ \sum_i\sum_j A_{ij} = \frac{1}{s^2}\sum_k\sum_l A_{kl}'$$

In [ ]:
np.sum(image), np.sum(scaledimage)*(1/0.5)**2.

In [ ]:
NN = 300
xx=np.arange(0,NN,1)
counts = np.sqrt(xx)
image = counts*counts[:,None]
scaledimage = geometric_transform(image,mapping,extra_keywords={'s':0.5})

fig,axs=plt.subplots(ncols=2)
axs[0].imshow(image)
axs[1].imshow(scaledimage)
plt.show()

In [ ]:
np.sum(image), np.sum(scaledimage)*(1/0.5)**2.

# How many counts are lost in general?
The above example shows that the geometric transformation is not perfectly flux-conserving, even though it might be for particular images and particular on-sky geometries (e.g. symmetrical sources).

To understand the extent to which image scaling affects flux conservation, I plot below the distribution of differences
$$ d=\frac{\sum_i\sum_j A_{ij} - \frac{1}{s^2}\sum_k\sum_l A_{kl}' }{\sum_i\sum_j A_{ij}} $$
between raw images and redshift-scaled images for a subsample of 300 ECO galaxy groups.

In [ ]:
import os
from astropy.io import fits
from random import sample
from scipy.stats import ks_2samp as kstest, anderson_ksamp as adtest
import pandas as pd

In [ ]:
fnames=os.listdir('./g3rassimages/broad/eco_broad_cts_ptsrc_rm')
groupfile = pd.read_csv("../g3groups/ECOdata_G3catalog_luminosity.csv")
groupfile = groupfile[groupfile.g3fc_l==1.0]

In [ ]:
diff=[]
pval=[]
zscale=[]
for ff in fnames[0:5]:
    grpid=float(ff.split('_')[2][3:-5])
    czval = groupfile[['g3grpcz_l']][groupfile.g3grp_l==grpid]
    scalefactor = np.array(czval/7470)[0]
    original=fits.open('./g3rassimages/broad/eco_broad_cts_ptsrc_rm/'+ff)[0].data
    rescaled=fits.open('./g3rassimages/broad/eco_broad_cts_scaled/'+ff)[0].data
    original = original.flatten()
    rescaled = (1/scalefactor**2.)*rescaled.flatten()
    if True:
        binv=np.arange(0,20,1)
        plt.figure()
        plt.hist(original,bins=binv)
        plt.hist(rescaled,bins=binv,histtype='step',linewidth=2)
        plt.yscale('log')
        plt.show()
    totalcounts=np.sum(original)
    rescaledcounts=np.sum(rescaled)
    diff.append((totalcounts-rescaledcounts)/totalcounts)
    zscale.append(scalefactor)

In [ ]:
fig,axs=plt.subplots(ncols=2)
axs[0].scatter(np.array(zscale)*7470.,diff)
axs[0].set_xlabel("original cz")
axs[0].set_ylabel("Fraction of Original Counts Gained\nor Lost in Rescaled Image")

In [ ]:
diff=[]
ksval=[]
cntr=1
ss=np.random.uniform(2530,7470,1000)/7470.
files = sample(os.listdir('g3rassimages/broad/eco_broad_cts/'),10)
for index,ff in enumerate(files):
    if 'Cnt' in ff:
        image = fits.open('./g3rassimages/broad/eco_broad_cts/'+ff)[0].data
        scaledimage = geometric_transform(image,mapping,cval=-99.,extra_keywords={'s':ss[index]})
        totalcounts=np.sum(image)
        scaledimage=scaledimage[scaledimage>=0]
        diff.append((totalcounts - (1/(ss[index]*ss[index]))*np.sum(scaledimage))/totalcounts)
        
        image= image.flatten()
        scaledimage = (1/(ss[index]*ss[index]))*scaledimage[scaledimage>-99].flatten()
        print(kstest(image,scaledimage))
#         plt.figure()
#         _,bv,_=plt.hist(image,log=True,bins=15)
#         plt.hist(scaledimage,log=True,histtype='step',bins=bv)
#         plt.show()
        
        cntr+=1
        if cntr==101: break

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(diff,bins='fd', histtype='stepfilled', color='teal')
median=np.median(diff)
plt.axvline(median, label='Median = {:0.2E}'.format(median), color='k')
plt.xlabel("Fraction of Counts in Original Image\n Gained or Lost in Resaled Image", fontsize=12)
plt.title("Sample of {} Count Maps".format(len(diff)),fontsize=12)
plt.legend(loc='best',fontsize=11)
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(ss[0:len(diff)]*7470., diff, s=2, alpha=1)
plt.xlabel("original cz value")
plt.ylabel("Fraction of Counts Gained in \n Rescaled Image")
plt.title("Sample of {} Count Maps".format(len(diff)),fontsize=12)
plt.show()

# How much exposure time is lost in general?

In [ ]:
diff=[]
cntr=1
for ff in os.listdir('g3rassimages/eco_broad/'):
    if 'Exp' in ff:
        image = fits.open('./g3rassimages/eco_broad/'+ff)[0].data
        ss=0.5
        scaledimage = geometric_transform(image,mapping,extra_keywords={'s':ss})
        totalcounts=np.sum(image)
        diff.append((totalcounts - (1/(ss*ss))*np.sum(scaledimage))/totalcounts)
        cntr+=1
        if cntr==501: break

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(diff,bins='fd', histtype='stepfilled', color='teal')
median=np.median(diff)
plt.axvline(median, label='Median = {:0.2E}'.format(median), color='k')
plt.xlabel("\n$d$: Fraction of Time in Original Image\n Gained or Lost in Scaled Image", fontsize=12)
plt.title("Sample of {} Count Maps".format(len(diff)),fontsize=12)
plt.legend(loc='best',fontsize=11)
plt.show()